# Khám phá dữ liệu, tiền xử lý

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

### Đọc dữ liệu từ file

In [2]:
air_df = pd.read_csv('data.csv', sep = ',')

In [3]:
air_df.head()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi
0,1121.52,2.49,15.59,6.62,7.03,51.11,60.88,6.21,5
1,767.71,1.48,12.00,32.90,8.46,36.26,42.39,3.52,4
2,727.65,1.41,10.11,51.50,8.35,35.83,41.03,3.23,4
3,727.65,1.03,8.82,77.25,8.94,39.41,44.50,3.58,4
4,761.03,0.59,7.37,103.00,9.18,46.39,51.56,3.99,4


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [4]:
air_df.shape

(13206, 9)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong DataFrame `air_df` cho biết dữ liệu về các chỉ số và chất lượng không khí của các tỉnh, thành phố từ 0h ngày 1/8/2021 - 0h ngày 1/9/2021 . Có vẻ không có vấn đề các dòng có ý nghĩa khác nhau.

### Dữ liệu có các dòng bị lặp không?

In [5]:
air_df.index.duplicated().sum()

0

### Mỗi cột có ý nghĩa gì?

In [6]:
with open('description.txt', 'r',encoding='utf-8') as f:
    print(f.read())

VARIABLE DESCRIPTIONS:
aqi        	Chỉ số chất lượng không khí (1,2,3,4,5)
                (1 = Good; 2 = Fair; 3 = Moderate; 4 = Poor ; 5 = Very Poor )
co          	Nồng độ CO (Carbon monoxide) 		,  μg/m3
no            	Nồng độ NO (Nitrogen monoxide) 		,  μg/m3
no2             Nồng độ NO2 (Nitrogen dioxide) 		,  μg/m3
o3             	Nồng độ O3 (Ozone) 			,  μg/m3
so2           	Nồng độ SO2 (Sulphur dioxide) 		,  μg/m3
pm2_5           Nồng độ PM2.5 (Fine particles matter) 	,  μg/m3
pm10          	Nồng độ PM10 (Coarse particulate matter),  μg/m3
nh3            	Nồng độ NH3 (Ammonia) 			,  μg/m3




## Đưa ra câu hỏi cần trả lời

- Chất lượng không khí được tính như thế nào từ các chỉ số trong không khí đã được đo từ môi trường ?
<br/>
- Ý nghĩa thực tế của câu hỏi: Theo Tổ chức Y tế thế giới (WHO), hằng năm có 34.000 ca tử vong tại Việt Nam có liên quan đến ô nhiễm không khí. Việc xác định chất lượng không khí giúp người dân có các biện pháp bảo vệ bản thân và cộng đồng kịp thời như đeo khẩu trang, hạn chế sử dụng các phương tiện giao thông gây ô nhiễm, ... . Ngoài ra, chính quyền địa phương có thể hạn chế lượng khí thải từ các nhà máy, xí nghiệp, trồng thêm nhiều cây xanh để làm giảm ô nhiễm không khí.

# Khám phá dữ liệu

In [7]:
# Cột output hiện có kiểu dữ liệu gì?
air_df['aqi'].dtype

dtype('int64')

In [8]:
# Cột output có bao nhiêu giá trị thiếu?
air_df['aqi'].isna().sum()

0

In [9]:
# Tỉ lệ các lớp trong cột output?
air_df['aqi'].value_counts(normalize=True) * 100

1    31.909738
2    26.344086
4    22.088445
5    11.252461
3     8.405270
Name: aqi, dtype: float64

- Cột output đang ở kiểu dạng số, không có giá trị thiếu và tỉ lệ lớp 3 và 5 trong cột output chênh lệch nhiều so với các lớp còn lại, vì vậy hãy lấy random khoảng 40% lớp 1, 2, 4 để các lớp có tỉ lệ gần nhau nhất

In [32]:
aqi_124 = [1, 2, 4]
frames = []

for aqi in aqi_124:
    df = air_df.loc[air_df['aqi'] == aqi]
    frames.append(df.sample(frac = 0.4))
    
aqi_35 = [3, 5]
for aqi in aqi_35:
    df = air_df.loc[air_df['aqi'] == aqi]
    frames.append(df)
    
air_df_bal = pd.concat(frames)
air_df_bal

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi
9888,397.21,0.06,0.63,23.96,0.14,0.51,0.75,0.39,1
6812,313.76,0.00,4.67,9.57,3.46,6.59,7.72,0.44,1
10569,263.69,0.00,1.20,21.99,0.20,0.73,0.91,0.30,1
11252,487.33,1.96,19.54,18.06,17.41,8.97,11.52,5.32,1
8974,201.94,0.06,0.66,22.17,0.16,0.50,0.66,0.19,1
...,...,...,...,...,...,...,...,...,...
13201,2616.88,59.01,30.16,0.00,38.62,138.76,152.62,14.19,5
13202,2590.18,64.37,29.47,0.00,41.96,144.76,158.74,14.31,5
13203,2937.32,83.15,33.59,0.00,55.31,174.44,189.82,15.07,5
13204,3471.37,109.97,42.50,0.00,73.43,219.32,237.25,14.69,5


- Kiểm tra lại 1 lần nữa tỉ lệ các giá trị ouput

In [33]:
air_df_bal['aqi'].value_counts(normalize=True) * 100

1    24.645520
5    21.721970
2    20.347902
4    17.058910
3    16.225698
Name: aqi, dtype: float64

## Tiền xử lý (tách các tập)

In [12]:
# Tách X và y
y_sr = air_df_bal["aqi"] # sr là viết tắt của series
X_df = air_df_bal.drop("aqi", axis=1)

In [13]:
# Tách tập huấn luyện,tập validation, tập test theo tỉ lệ 70%:20%:10%
temp_X_df, test_X_df, temp_y_sr, test_y_sr = \
                              train_test_split(X_df, y_sr, 
                                               test_size=0.1, 
                                               stratify=y_sr, 
                                               random_state=60)
train_X_df, val_X_df, train_y_sr, val_y_sr = \
                              train_test_split(temp_X_df, temp_y_sr, 
                                               test_size=2/9, 
                                               stratify=temp_y_sr, 
                                               random_state=60)


In [14]:
train_X_df.shape

(4788, 8)

In [15]:
train_y_sr.shape

(4788,)

In [16]:
val_X_df.shape

(1368, 8)

In [17]:
val_y_sr.shape

(1368,)

In [18]:
test_X_df.shape

(685, 8)

In [19]:
test_y_sr.shape

(685,)

In [20]:
train_X_df.head().index

Int64Index([12699, 10251, 13, 7339, 7550], dtype='int64')

# Khám phá dữ liệu (tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [21]:
train_X_df.dtypes

co       float64
no       float64
no2      float64
o3       float64
so2      float64
pm2_5    float64
pm10     float64
nh3      float64
dtype: object

- Các cột đều có dữ liệu phù hợp

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [22]:
def missing_percentage(c):
    return (c.isna().mean() * 100).round(1)
def median(c):
    return c.quantile(0.5).round(1)
train_X_df.agg([missing_percentage, 'min', median, 'max'])

,co,no,no2,o3,so2,pm2_5,pm10,nh3
missing_percentage,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
min,166.89,0.00,0.20,0.00,0.05,0.50,0.54,0.00
median,527.40,0.10,7.20,22.40,3.70,21.30,24.30,2.00
max,14953.61,343.32,135.72,223.16,225.07,811.92,844.52,102.34


## Tiền xử lý (tập huấn luyện)

- Với các cột dạng số, ta sẽ điền giá trị thiếu bằng giá trị mean của cột <font color=gray>(gợi ý: dùng `SimpleImputer` trong Sklearn)</font>. Với *tất cả* các cột dạng số trong tập huấn luyện, ta đều cần tính mean, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới. 

- Cuối cùng, khi tất cả các cột đã được điền giá trị thiếu và đã có dạng số, ta sẽ tiến hành chuẩn hóa bằng cách trừ đi mean và chia cho độ lệch chuẩn của cột để giúp cho các thuật toán cực tiểu hóa như Gradient Descent, LBFGS, ... hội tụ nhanh hơn <font color=gray>(gợi ý: dùng `StandardScaler` trong Sklearn)</font>.

In [23]:
cols=list(train_X_df.columns)
temp=make_column_transformer(
        (SimpleImputer(strategy='mean'),cols))
preprocess_pipeline=make_pipeline(temp,StandardScaler())
preprocessed_train_X=preprocess_pipeline.fit_transform(train_X_df)

In [24]:
preprocess_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(),
                                                  ['co', 'no', 'no2', 'o3',
                                                   'so2', 'pm2_5', 'pm10',
                                                   'nh3'])])),
                ('standardscaler', StandardScaler())])

## Tiền xử lý (tập validation)

In [25]:
preprocessed_val_X=preprocess_pipeline.transform(val_X_df)
preprocessed_val_X.shape

(1368, 8)

## Tiền xử lý + mô hình hóa
### Tìm mô hình tốt nhất

In [26]:
neural_net_model = MLPClassifier(hidden_layer_sizes=(20,), activation='tanh',\
                                 solver='lbfgs', random_state=60, max_iter=2500)

full_pipeline = make_pipeline(preprocess_pipeline, neural_net_model)

# Thử nghiệm với các giá trị khác nhau của các siêu tham số
# và chọn ra các giá trị tốt nhất
train_errs = []
val_errs = []
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
best_val_err = float('inf')
best_alpha = None
for alpha in alphas:
    full_pipeline.set_params(mlpclassifier__alpha = alpha)

    full_pipeline.fit(train_X_df, train_y_sr)
    train_errs.append((1 - full_pipeline.score(train_X_df, train_y_sr)) * 100)
    val_errs.append((1 - full_pipeline.score(val_X_df, val_y_sr)) * 100)

best_val_err = min(val_errs)
index = val_errs.index(min(val_errs))
best_alpha = alphas[index]

'Finish!'

'Finish!'

In [27]:
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(transformers=[('simpleimputer',
                                                                   SimpleImputer(),
                                                                   ['co', 'no',
                                                                    'no2', 'o3',
                                                                    'so2',
                                                                    'pm2_5',
                                                                    'pm10',
                                                                    'nh3'])])),
                                 ('standardscaler', StandardScaler())])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', alpha=10,
                               hidden_layer_sizes=(20,), max_iter=2500,
                               random_state=60, solver='lbfgs'))])

In [28]:
print('best_val_err: ', best_val_err.round(3))
print()
print('best_alpha: ',best_alpha)
print()
print('train error: ', train_errs)
print()
print('valid error: ', val_errs)

best_val_err:  0.658

best_alpha:  0.001

train error:  [0.0, 0.0, 0.0, 0.22974101921470202, 1.670843776106934, 3.5923141186299135]

valid error:  [1.6812865497076057, 0.6578947368421018, 1.0964912280701733, 0.8771929824561431, 1.6812865497076057, 3.14327485380117]


Cuối cùng, huấn luyện lại `full_pipeline` trên `X_df` và `y_sr` (tập huấn luyện + tập validation) với `best_alpha` tìm được ở trên để ra được mô hình cụ thể cuối cùng.

In [29]:
full_pipeline.set_params(mlpclassifier__alpha = best_alpha)

full_pipeline.fit(temp_X_df, temp_y_sr)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(transformers=[('simpleimputer',
                                                                   SimpleImputer(),
                                                                   ['co', 'no',
                                                                    'no2', 'o3',
                                                                    'so2',
                                                                    'pm2_5',
                                                                    'pm10',
                                                                    'nh3'])])),
                                 ('standardscaler', StandardScaler())])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', alpha=0.001,
                               hidden_layer_sizes=(20,), max_iter=2500,
                               random_state=60, solver='lbfgs'))])

### Đánh giá mô hình tìm được:
Đánh giá mô hình tìm được với test data

In [30]:
score = (full_pipeline.score(test_X_df, test_y_sr) * 100).round(2)
print('Score with test data: ', score, '%')

Score with test data:  99.42 %
